In [83]:
import pandas as pd
import numpy as np
import scipy as sp
import os
from scipy.sparse import csr_matrix, lil_matrix, eye
from scipy.sparse.linalg import spsolve
from openpyxl import load_workbook
import requests
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [3]:
dir_name = os.path.join("..", "Data")
df_human = pd.read_csv(os.path.join(dir_name, "9606.protein.links.full.v12.0.txt"), sep=" ")
df_rvfv = pd.read_csv(os.path.join(dir_name, "string_interactions.tsv"), sep="\t")


In [ ]:
response = requests.get("https://string-db.org/api/tsv/interaction_partners?identifiers=ENSP00000335153")


In [4]:
df_human

,protein1,protein2,neighborhood,neighborhood_transferred,fusion,cooccurence,homology,coexpression,coexpression_transferred,experiments,experiments_transferred,database,database_transferred,textmining,textmining_transferred,combined_score
0,9606.ENSP00000000233,9606.ENSP00000356607,0,0,0,0,0,0,45,0,134,0,0,0,81,173
1,9606.ENSP00000000233,9606.ENSP00000427567,0,0,0,0,0,0,0,0,128,0,0,0,70,154
2,9606.ENSP00000000233,9606.ENSP00000253413,0,0,0,0,0,49,111,0,49,0,0,0,69,151
3,9606.ENSP00000000233,9606.ENSP00000493357,0,0,0,0,0,56,0,0,53,0,0,433,81,471
4,9606.ENSP00000000233,9606.ENSP00000324127,0,0,0,0,0,0,0,0,46,0,0,153,91,201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13715399,9606.ENSP00000501317,9606.ENSP00000475489,0,0,0,0,0,60,0,0,99,0,0,126,0,195
13715400,9606.ENSP00000501317,9606.ENSP00000370447,0,0,0,0,0,0,55,0,111,0,0,0,79,158
13715401,9606.ENSP00000501317,9606.ENSP00000312272,0,0,0,0,0,0,0,0,0,0,0,187,88,226
13715402,9606.ENSP00000501317,9606.ENSP00000402092,0,0,0,0,0,0,0,0,67,0,0,146,0,169


In [86]:
combined_scores = np.array(df_human['combined_score']).reshape(-1,1)
combined_scores

scaler = MinMaxScaler()
scaled_scores = scaler.fit_transform(combined_scores)


In [ ]:
scaled_scores

13715404

In [5]:
unique_prots = set(df_human["protein1"]) | set(df_human["protein2"])
#print(len(unique_prots))
unique_prots = list(unique_prots)
prot_map = {p:i for i,p in enumerate(unique_prots)}
prot_map


{'9606.ENSP00000356130': 0,
 '9606.ENSP00000356607': 1,
 '9606.ENSP00000184183': 2,
 '9606.ENSP00000263257': 3,
 '9606.ENSP00000319486': 4,
 '9606.ENSP00000329312': 5,
 '9606.ENSP00000332756': 6,
 '9606.ENSP00000347418': 7,
 '9606.ENSP00000429744': 8,
 '9606.ENSP00000295522': 9,
 '9606.ENSP00000281419': 10,
 '9606.ENSP00000334798': 11,
 '9606.ENSP00000383558': 12,
 '9606.ENSP00000309376': 13,
 '9606.ENSP00000389841': 14,
 '9606.ENSP00000296632': 15,
 '9606.ENSP00000354335': 16,
 '9606.ENSP00000414150': 17,
 '9606.ENSP00000375429': 18,
 '9606.ENSP00000466829': 19,
 '9606.ENSP00000378974': 20,
 '9606.ENSP00000343172': 21,
 '9606.ENSP00000297767': 22,
 '9606.ENSP00000384117': 23,
 '9606.ENSP00000262210': 24,
 '9606.ENSP00000211936': 25,
 '9606.ENSP00000263238': 26,
 '9606.ENSP00000376410': 27,
 '9606.ENSP00000357789': 28,
 '9606.ENSP00000221307': 29,
 '9606.ENSP00000269576': 30,
 '9606.ENSP00000303153': 31,
 '9606.ENSP00000328182': 32,
 '9606.ENSP00000457492': 33,
 '9606.ENSP00000386126':

In [6]:
# Generate list of unique host proteins that interact with NSs
rvfv_prot_list = set(pd.concat([df_rvfv["node1_external_id"], df_rvfv["node2_external_id"]], axis=0, ignore_index=True))
rvfv_interactor_list = {p for p in rvfv_prot_list if "9606.ENSP" in p}


# Create dictionary mapping NSs interactor protein names to their array index
s_array = np.zeros(len(unique_prots))
for prot in rvfv_interactor_list:
    if prot in prot_map:
        s_array[prot_map[prot]] = 1.0
    else:
        print(f'Protein {prot} does not interact with any human proteins.')


Protein 9606.ENSP00000360286 does not interact with any human proteins.
Protein 9606.ENSP00000356348 does not interact with any human proteins.
Protein 9606.ENSP00000384144 does not interact with any human proteins.
Protein 9606.ENSP00000353622 does not interact with any human proteins.


In [7]:
rvfv_interactor_list

{'9606.ENSP00000265651',
 '9606.ENSP00000335153',
 '9606.ENSP00000353622',
 '9606.ENSP00000356348',
 '9606.ENSP00000360154',
 '9606.ENSP00000360286',
 '9606.ENSP00000369581',
 '9606.ENSP00000377958',
 '9606.ENSP00000384144',
 '9606.ENSP00000451560'}

In [8]:
s_array.sum()


6.0

In [9]:
# Pair human protein columns, map protein names to matrix coordinates, change matrix value to one at that position (and inverse position)
w_sparse_matrix = lil_matrix((len(prot_map), len(prot_map)))

for prot1, prot2 in zip(df_human['protein1'], df_human['protein2']):
    w_sparse_matrix[prot_map[prot1], prot_map[prot2]] = 1
    w_sparse_matrix[prot_map[prot2], prot_map[prot1]] = 1

w_sparse_matrix = w_sparse_matrix.tocsr()




In [10]:
D = np.sqrt(w_sparse_matrix.sum(-1))
D[D < 1] = 1
w_sparse_matrix = w_sparse_matrix.multiply(1.0/D)
w_sparse_matrix = w_sparse_matrix.multiply(1.0/D.T)

#w_sparse_matrix = w_sparse_matrix / D / D.T

In [11]:
alpha = 0.5
M = (1 + alpha) * eye(len(unique_prots)) - (w_sparse_matrix * alpha)

In [12]:
#y_array = spsolve(M,s_array)

In [13]:
# Build a function that trims the network to a certain number of rounds of connection
def select_network_subset(starting_nodes, full_network, iterations, s_array=None):
    if s_array is None:
        s_array = starting_nodes
    
    # Build out network for given number of cycles
    if iterations > 0:
        new_starting_nodes = (full_network@starting_nodes) + starting_nodes
        return select_network_subset(new_starting_nodes, full_network, iterations - 1, s_array=s_array)
    
    else:
        starting_nodes[starting_nodes > 0.0] = 1.0
        print(f"Starting nodes: {starting_nodes.sum()}")
        
        trimmed_network_list = [p for p,m in zip(unique_prots, starting_nodes) if m > 0]

        starting_mask = starting_nodes > 0

        trimmed_network = full_network[starting_mask,:][:,starting_mask]

        trimmed_s_array = s_array[starting_mask]
        

        return trimmed_s_array, trimmed_network, trimmed_network_list


        

In [14]:
trimmed_s_array, trimmed_network, trimmed_network_list = select_network_subset(s_array, w_sparse_matrix.tocsr(), 1)

Starting nodes: 7837.0


In [ ]:
# Run algorithm 

def calculate_scores(s_array, network, network_list, d_or_i='D', alpha=0.5):
    D = np.sqrt(network.sum(-1))
    D[D < 1] = 1

    normalized_matrix = network.multiply(1.0/D)
    normalized_matrix = normalized_matrix.multiply(1.0/D.T)

    I = eye(len(network_list))

    if d_or_i.lower() == 'd':
        D_tilde = normalized_matrix.sum(axis=1).A1
        D_tilde = csr_matrix(np.matrix(np.diag(D_tilde)))   
        L = D_tilde - normalized_matrix

    elif d_or_i.lower() == 'i':
        L = I - normalized_matrix

    else:
        print(f'{d_or_i} is an improper input for Laplacian calculation. Select D or I.')

    M = I + (alpha * L)
    y_array = spsolve(M, s_array)

    return y_array


In [ ]:
# Pair scores with protein names and filter

def sort_filter_scores(y_array, network_list, rvfv_interactor_list):
    prots_scores_zipped = list(zip(y_array.tolist(), network_list))
    prots_scores_sorted = sorted(prots_scores_zipped, key=lambda x: x[0], reverse=True)
    prots_scores_filtered = [tup for tup in prots_scores_sorted if tup[1] not in rvfv_interactor_list]

    return prots_scores_filtered

In [19]:
# I and one-step network

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 1)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='I', alpha=0.5)

prot_scores_I_one = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list)

Starting nodes: 7837.0


In [20]:
# D tilde and one-step network

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 1)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='D', alpha=0.5)

prot_scores_D_one = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list)

Starting nodes: 7837.0


In [21]:
# I and full network

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 3)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='I', alpha=0.5)

prot_scores_I_full = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list)

Starting nodes: 19622.0


In [22]:
# D tilde and full network

s_array_one, network_one, network_list_one = select_network_subset(s_array, w_sparse_matrix.tocsr(), 3)
y_array_one = calculate_scores(s_array_one, network_one, network_list_one, d_or_i='D', alpha=0.5)

prot_scores_D_full = sort_filter_scores(y_array_one, network_list_one, rvfv_interactor_list)

Starting nodes: 19622.0


In [ ]:
# Add scores to dataframe

scores_df = pd.DataFrame(index=[p for _,p in prot_scores_I_one], columns=["I_one"])

scores = [
    (prot_scores_I_one, 'I_one'),
    (prot_scores_I_full, 'I_full'),
    (prot_scores_D_one, 'D_one'),
    (prot_scores_D_full, 'D_full'),
]

for prot_scores, column_name in scores:
    for s, p in prot_scores:
        scores_df.loc[p, column_name] = s

In [ ]:
# Add rank columns for each run

rank_columns = ['I_one', 'I_full', 'D_one', 'D_full']

for col in rank_columns:
    scores_df = scores_df.sort_values(col, ascending=False)
    scores_df[f'{col}_rank'] = list(range(1, len(scores_df) + 1))

In [90]:
scores_df = scores_df.sort_values('D_full', ascending=False)
scores_df[:15]

,I_one,I_full,D_one,D_full,I_one_rank,I_full_rank,D_one_rank,D_full_rank
9606.ENSP00000355432,0.001405,0.001405,0.002123,0.001924,1,1,1,1
9606.ENSP00000368296,0.001326,0.001328,0.001932,0.001799,2,2,2,2
9606.ENSP00000452245,0.001311,0.001312,0.001779,0.001627,3,3,3,3
9606.ENSP00000453012,0.000921,0.000923,0.001352,0.001203,8,8,4,4
9606.ENSP00000263071,0.00107,0.001070,0.001347,0.001193,4,4,5,5
9606.ENSP00000292853,0.000963,0.000963,0.001318,0.001192,7,7,6,6
9606.ENSP00000357588,0.000894,0.000896,0.001310,0.001155,10,10,7,7
9606.ENSP00000229330,0.000965,0.000964,0.001221,0.001115,6,6,8,8
9606.ENSP00000356355,0.001005,0.001003,0.001191,0.001083,5,5,9,9
9606.ENSP00000281623,0.000912,0.000910,0.001142,0.001058,9,9,11,10


63